In [11]:
from models.pyramid import PyramidModule, ConvolutionalResBlock
import torch
import torchsummary 
pyramid_module = PyramidModule(N=2, M=2,inshape=(19,9,9))
torchsummary.summary(pyramid_module, (19, 9, 9), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 256, 9, 9]          44,032
              ReLU-2            [-1, 256, 9, 9]               0
            Conv2d-3            [-1, 128, 9, 9]         294,912
         GroupNorm-4            [-1, 128, 9, 9]             256
              ReLU-5            [-1, 128, 9, 9]               0
            Conv2d-6            [-1, 256, 9, 9]         294,912
         GroupNorm-7            [-1, 256, 9, 9]             512
              ReLU-8            [-1, 256, 9, 9]               0
ConvolutionalResBlock-9  [[-1, 256, 9, 9], [-1, 256, 9, 9]]               0
           Conv2d-10            [-1, 128, 9, 9]         294,912
        GroupNorm-11            [-1, 128, 9, 9]             256
             ReLU-12            [-1, 128, 9, 9]               0
           Conv2d-13            [-1, 256, 9, 9]         294,912
        GroupNorm-14       

In [1]:
from models.fan_init_pyramid import PyramidModule, ConvolutionalResBlock
import torch
import torchsummary 
pyramid_module = PyramidModule(N=2, M=2,inshape=(19,9,9))
torchsummary.summary(pyramid_module, (19, 9, 9), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 256, 9, 9]          44,032
              ReLU-2            [-1, 256, 9, 9]               0
         GroupNorm-3            [-1, 256, 9, 9]             512
            Conv2d-4            [-1, 128, 9, 9]         294,912
FanInInitReLULayer-5            [-1, 128, 9, 9]               0
         GroupNorm-6            [-1, 128, 9, 9]             256
            Conv2d-7            [-1, 256, 9, 9]         294,912
FanInInitReLULayer-8            [-1, 256, 9, 9]               0
ConvolutionalResBlock-9  [[-1, 256, 9, 9], [-1, 256, 9, 9]]               0
        GroupNorm-10            [-1, 256, 9, 9]             512
           Conv2d-11            [-1, 128, 9, 9]         294,912
FanInInitReLULayer-12            [-1, 128, 9, 9]               0
        GroupNorm-13            [-1, 128, 9, 9]             256
           Conv2d-14      

In [7]:
from models.fan_init_pyramid import PyramidModule, ConvolutionalResBlock
import torch
import torchsummary 
pyramid_module = PyramidModule(N=2, M=2,inshape=(19,9,9)).to('cuda')
inp = torch.rand(1, 19, 9, 9).to('cuda'  )
with torch.autograd.profiler.profile(use_cuda=True) as prof:
   for _ in range(100):
      out = pyramid_module(inp)
print(prof)

cuda:0
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        cudaEventRecord         0.06%      20.000us         0.06%      20.000us      20.000us       0.000us         0.00%       0.000us       0.000us             1  
                                           aten::conv2d         0.15%      46.000us         1.62%     505.000us     505.000us      59.000us         0.19%     521.000us     521.000us             1  
  

STAGE:2024-02-18 14:11:57 3557529:3557529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-02-18 14:11:57 3557529:3557529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-02-18 14:11:57 3557529:3557529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [3]:
pyramid_module(torch.rand(1, 19, 9, 9))

tensor([[1.4866, 0.8184, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)

torch.Size([2, 160, 5, 5]) torch.Size([2, 320, 5, 5])
torch.Size([2, 128, 5, 5]) torch.Size([2, 256, 5, 5])
torch.Size([2, 128, 9, 9]) torch.Size([2, 256, 5, 5])
torch.Size([2, 128, 9, 9]) torch.Size([2, 256, 9, 9])
torch.Size([2, 128, 9, 9]) torch.Size([2, 256, 9, 9])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 256, 9, 9]         230,656
              ReLU-2            [-1, 256, 9, 9]               0
            Conv2d-3            [-1, 128, 9, 9]         294,912
         GroupNorm-4            [-1, 128, 9, 9]             256
              ReLU-5            [-1, 128, 9, 9]               0
            Conv2d-6            [-1, 256, 9, 9]         294,912
         GroupNorm-7            [-1, 256, 9, 9]             512
              ReLU-8            [-1, 256, 9, 9]               0
ConvolutionalResBlock-9  [[-1, 256, 9, 9], [-1, 256, 9, 9]]               0
           Co

In [7]:
from models.fan_init_pyramid import ConvolutionalResBlock
import torchsummary
import torch
res_block = ConvolutionalResBlock(19, 128, 3)
torchsummary.summary(res_block, (19, 9, 9), device='cpu')

o1, o2 = res_block(torch.randn(1, 19, 9, 9))
print(o1.shape, o2.shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         GroupNorm-1             [-1, 19, 9, 9]              38
            Conv2d-2             [-1, 64, 3, 3]          10,944
FanInInitReLULayer-3             [-1, 64, 3, 3]               0
         GroupNorm-4             [-1, 64, 3, 3]             128
            Conv2d-5            [-1, 128, 3, 3]          73,728
FanInInitReLULayer-6            [-1, 128, 3, 3]               0
         GroupNorm-7             [-1, 19, 9, 9]              38
            Conv2d-8            [-1, 128, 3, 3]           2,432
FanInInitReLULayer-9            [-1, 128, 3, 3]               0
        GroupNorm-10             [-1, 19, 9, 9]              38
           Conv2d-11             [-1, 19, 3, 3]             361
FanInInitReLULayer-12             [-1, 19, 3, 3]               0
Total params: 87,707
Trainable params: 87,707
Non-trainable params: 0
--------------------------------

In [5]:
from models.pyramid import ConvolutionalResBlock
import torchsummary
import torch
res_block = ConvolutionalResBlock(19, 128, 3)
torchsummary.summary(res_block, (19, 9, 9), device='cpu')

o1, o2 = res_block(torch.randn(1, 19, 9, 9))
print(o1.shape, o2.shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 3, 3]          10,944
         GroupNorm-2             [-1, 64, 3, 3]             128
              ReLU-3             [-1, 64, 3, 3]               0
            Conv2d-4            [-1, 128, 3, 3]          73,728
         GroupNorm-5            [-1, 128, 3, 3]             256
            Conv2d-6            [-1, 128, 3, 3]           2,432
         GroupNorm-7            [-1, 128, 3, 3]             256
              ReLU-8            [-1, 128, 3, 3]               0
            Conv2d-9             [-1, 19, 3, 3]             361
Total params: 88,105
Trainable params: 88,105
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.34
Estimated Total Size (MB): 0.40
---------------------------------------------

In [8]:
from models.fan_init_pyramid import ConvolutionalResBlock, DeconvolutionalResBlock
import torchsummary
import torch
res_block = DeconvolutionalResBlock(19, 128, 3)
#torchsummary.summary(res_block, (19, 9, 9), device='cpu')

o1 = res_block(torch.randn(1, 19, 9, 9), torch.randn(1, 128, 9, 9))
print(o1.shape)

torch.Size([1, 128, 25, 25])


In [9]:
from models.pyramid import ConvolutionalResBlock, DeconvolutionalResBlock
import torchsummary
import torch
res_block = DeconvolutionalResBlock(19, 128, 3)
#torchsummary.summary(res_block, (19, 9, 9), device='cpu')

o1 = res_block(torch.randn(1, 19, 9, 9), torch.randn(1, 128, 9, 9))
print(o1.shape)

torch.Size([1, 128, 25, 25])
